# Loomaeksperdisüsteemi rakendamine

Näide [AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners).

Selles näites rakendame lihtsat teadmistepõhist süsteemi, et tuvastada loom mõnede füüsikaliste omaduste põhjal. Süsteemi saab esitada järgmise JA-VÕI-puuna (see on osa kogu puust, me võime hõlpsasti lisada veel reegleid):

![](../../../../../../translated_images/et/AND-OR-Tree.5592d2c70187f283.webp)


## Meie oma eksperdisüsteemide kest tagurpidi järeldusega

Proovime defineerida lihtsat teadmiste esindamise keelt, mis põhineb tootmisreeglitel. Kasutame reeglite määratlemiseks märksõnadena Python'i klasse. Tegelikkuses on kolm klassi tüüpi:
* `Ask` esindab küsimust, mida tuleb kasutajalt küsida. See sisaldab võimalike vastuste komplekti.
* `If` esindab reeglit ja on lihtsalt sünteetiline mugavus reegli sisu salvestamiseks
* `AND`/`OR` on klassid, mis esindavad puu JA/VOI harusid. Need salvestavad lihtsalt sisemuses argumentide nimekirja. Koodi lihtsustamiseks on kogu funktsionaalsus defineeritud vanemklassina `Content`


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

Meie süsteemis sisaldaks töömälus nimekirja **faktidest** kui **atribuudi-väärtuse paaridest**. Teadmusbaas võib olla defineeritud kui üks suur sõnastik, mis seostab tegevused (uued faktid, mis tuleks lisada töömällu) tingimustega, väljendatuna JA-VÕI avaldistena. Samuti võib mõned fakti `Küsimuse` alla panna.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

Tagurpidi järelduse tegemiseks määratleme klassi `Knowledgebase`. See sisaldab:
* Töömälu `memory` — sõnastik, mis seob atribuute väärtustega
* Teadmistebaasi reeglid `rules` eelpool määratletud vormingus

Kaks peamist meetodit on:
* `get`, mis tagastab atribuudi väärtuse, vajadusel tehes järeldusi. Näiteks `get('color')` tagastab värvi atribuudi väärtuse (küsib vajadusel ja salvestab väärtuse hilisemaks kasutamiseks töömällu). Kui küsime `get('color:blue')`, siis küsitakse värvi ja tagastatakse vastus `y`/`n` sõltuvalt värvist.
* `eval` teostab tegelikku järeldamist, s.t läbib JA/VOI puu, hindab alam-eesmärke jne.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Nüüd defineerime oma loomade teadmistebaasi ja teostame konsultatsiooni. Pange tähele, et see kõne esitab teile küsimusi. Võite vastata tippides `y`/`n` jah-ei küsimustele või määrates numbri (0..N) küsimustele, millel on pikemad mitmikvastuse valikud.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Eksperta kasutamine edasisuunalise tuletamise jaoks

Järgmises näites proovime rakendada edasisuunalist tuletamist kasutades üht teadmiste esitamise teekidest, [Experta](https://github.com/nilp0inter/experta). **Experta** on teek edasisuunaliste tuletamissüsteemide loomiseks Pythonis, mis on loodud klassikalise vana süsteemi [CLIPS](http://www.clipsrules.net/index.html) sarnaseks.

Me oleksime võinud forward chaining moodustada ka ise ilma suuremate probleemideta, kuid lihtsad implementeeringud ei ole tavaliselt eriti tõhusad. Reeglite efektiivsemaks sobitamiseks kasutatakse spetsiaalset algoritmi [Rete](https://en.wikipedia.org/wiki/Rete_algorithm).


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

Me määratleme oma süsteemi klassina, mis pärib `KnowledgeEngine`. Iga reegel määratletakse eraldi funktsioonina koos `@Rule` annotatsiooniga, mis täpsustab, millal reegel peaks käivituma. Reegli sees saame uusi fakte lisada funktsiooni `declare` abil, ja nende faktide lisamine toob kaasa selle, et edasiviiva järeldusmootori poolt kutsutakse välja veel teisi reegleid.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Kui oleme määratlenud teadmistebaasi, täidame oma töömälu mõningate algandmetega ja seejärel kutsume välja meetodi `run()`, et teha järeldusi. Nagu tulemusena näete, lisatakse töömälu uued järeldatud faktid, sealhulgas lõplik fakt loomade kohta (kui oleme kõik algandmed õigesti seadistanud).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Vastutusest loobumine**:
See dokument on tõlgitud kasutades tehisintellekti tõlketeenust [Co-op Translator](https://github.com/Azure/co-op-translator). Kuigi püüame täpsust, palun pidage meeles, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Originaaldokument selle emakeeles tuleks pidada autoriteetseks allikaks. Olulise info puhul soovitatakse kasutada professionaalset inimtõlget. Me ei vastuta ühegi arusaamatuse või valesti tõlgendamise eest, mis võivad tekkida selle tõlke kasutamisel.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
